<a href="https://colab.research.google.com/github/tamara-kostova/IIS/blob/master/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LAB EXERCISE 4.1**

In [ ]:
!pip install torch
!pip install torch_geometric
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cpu.html


In [ ]:
import torch
from torch_geometric.nn import TransE, ComplEx


**Function for training**

In [ ]:
def train(model, data_loader, optimizer, epochs=50):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_examples = 0

        for head_index, rel_type, tail_index in data_loader:
            optimizer.zero_grad()
            loss = model.loss(head_index, rel_type, tail_index)
            loss.backward()
            optimizer.step()
            total_loss += float(loss) * head_index.numel()
            total_examples += head_index.numel()

        loss = total_loss / total_examples
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

**Function for evaluating**

In [ ]:
def evaluate(model, data_loader):
    hits1_list = []
    hits3_list = []
    hits10_list = []
    mr_list = []
    mrr_list = []

    for head_index, rel_type, tail_index in data_loader:
        head_embeds = model.node_emb(head_index)
        relation_embeds = model.rel_emb(rel_type)
        tail_embeds = model.node_emb(tail_index)

        if isinstance(model, TransE):
            scores = torch.norm(head_embeds + relation_embeds - tail_embeds, p=1, dim=1)

        elif isinstance(model, ComplEx):
            # Get real and imaginary parts
            re_relation, im_relation = torch.chunk(relation_embeds, 2, dim=1)
            re_head, im_head = torch.chunk(head_embeds, 2, dim=1)
            re_tail, im_tail = torch.chunk(tail_embeds, 2, dim=1)

            # Compute scores
            re_score = re_head * re_relation - im_head * im_relation
            im_score = re_head * im_relation + im_head * re_relation
            scores = (re_score * re_tail + im_score * im_tail)

            # Negate as we want to rank scores in ascending order, lower the better
            scores = - scores.sum(dim=1)

        else:
            raise ValueError(f'Unsupported model.')

        scores = scores.view(-1, head_embeds.size()[0])

        hits1, hits3, hits10, mr, mrr = eval_metrics(scores)
        hits1_list.append(hits1.item())
        hits3_list.append(hits3.item())
        hits10_list.append(hits10.item())
        mr_list.append(mr.item())
        mrr_list.append(mrr.item())

    hits1 = sum(hits1_list) / len(hits1_list)
    hits3 = sum(hits3_list) / len(hits1_list)
    hits10 = sum(hits10_list) / len(hits1_list)
    mr = sum(mr_list) / len(hits1_list)
    mrr = sum(mrr_list) / len(hits1_list)

    return hits1, hits3, hits10, mr, mrr

**Evaluation metrics**

In [ ]:
def eval_metrics(y_pred):
    argsort = torch.argsort(y_pred, dim=1, descending=False)
    # not using argsort to do the rankings to avoid bias when the scores are equal
    ranking_list = torch.nonzero(argsort == 0, as_tuple=False)
    ranking_list = ranking_list[:, 1] + 1
    hits1_list = (ranking_list <= 1).to(torch.float)
    hits3_list = (ranking_list <= 3).to(torch.float)
    hits10_list = (ranking_list <= 10).to(torch.float)
    mr_list = ranking_list.to(torch.float)
    mrr_list = 1. / ranking_list.to(torch.float)

    return hits1_list.mean(), hits3_list.mean(), hits10_list.mean(), mr_list.mean(), mrr_list.mean()

**Load data**

In [ ]:
from torch_geometric.datasets import FB15k_237

train_data = FB15k_237('../data/FB15k', split='train')[0]
val_data = FB15k_237('../data/FB15k', split='val')[0]
test_data = FB15k_237('../data/FB15k', split='test')[0]


## **EXERCISE 1**

TransE Knowledge Graph

In [ ]:
from torch.optim import Adam

model = TransE(num_nodes=train_data.num_nodes,
                   num_relations=train_data.num_edge_types,
                   hidden_channels=30)

loader = model.loader(head_index=train_data.edge_index[0],
                          rel_type=train_data.edge_type,
                          tail_index=train_data.edge_index[1],
                          batch_size=1000,
                          shuffle=True)

optimizer = Adam(model.parameters(), lr=0.01)

**Train model**

In [ ]:
train(model, loader, optimizer)

Epoch: 000, Loss: 0.7868
Epoch: 001, Loss: 0.5689
Epoch: 002, Loss: 0.4145
Epoch: 003, Loss: 0.3054
Epoch: 004, Loss: 0.2433
Epoch: 005, Loss: 0.2080
Epoch: 006, Loss: 0.1863
Epoch: 007, Loss: 0.1734
Epoch: 008, Loss: 0.1623
Epoch: 009, Loss: 0.1535
Epoch: 010, Loss: 0.1476
Epoch: 011, Loss: 0.1440
Epoch: 012, Loss: 0.1388
Epoch: 013, Loss: 0.1360
Epoch: 014, Loss: 0.1321
Epoch: 015, Loss: 0.1302
Epoch: 016, Loss: 0.1274
Epoch: 017, Loss: 0.1246
Epoch: 018, Loss: 0.1221
Epoch: 019, Loss: 0.1216
Epoch: 020, Loss: 0.1196
Epoch: 021, Loss: 0.1177
Epoch: 022, Loss: 0.1163
Epoch: 023, Loss: 0.1141
Epoch: 024, Loss: 0.1129
Epoch: 025, Loss: 0.1109
Epoch: 026, Loss: 0.1118
Epoch: 027, Loss: 0.1112
Epoch: 028, Loss: 0.1090
Epoch: 029, Loss: 0.1077
Epoch: 030, Loss: 0.1075
Epoch: 031, Loss: 0.1059
Epoch: 032, Loss: 0.1053
Epoch: 033, Loss: 0.1052
Epoch: 034, Loss: 0.1042
Epoch: 035, Loss: 0.1032
Epoch: 036, Loss: 0.1040
Epoch: 037, Loss: 0.1027
Epoch: 038, Loss: 0.1017
Epoch: 039, Loss: 0.1021


**TransE Model Results**

In [ ]:
hits1, hits3, hits10, mr, mrr = evaluate(model,loader)

In [ ]:
print(f'Mean Rank: {mr:.2f}, Mean Reciprocal Rank: {mrr:.4f}, '
      f'Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}')

Mean Rank: 532.97, Mean Reciprocal Rank: 0.0084, Hits@1: 0.0037, Hits@3: 0.0037, Hits@10: 0.0073


## **EXERCISE 2**

In [ ]:
model2 = ComplEx(num_nodes=train_data.num_nodes,
                   num_relations=train_data.num_edge_types,
                   hidden_channels=50)

loader2 = model2.loader(head_index=train_data.edge_index[0],
                          rel_type=train_data.edge_type,
                          tail_index=train_data.edge_index[1],
                          batch_size=1000,
                          shuffle=True)

optimizer2 = Adam(model2.parameters(), lr=0.01)

**Train model**

In [ ]:
train(model2, loader2, optimizer2)

Epoch: 000, Loss: 0.5524
Epoch: 001, Loss: 0.3052
Epoch: 002, Loss: 0.1696
Epoch: 003, Loss: 0.1295
Epoch: 004, Loss: 0.1107
Epoch: 005, Loss: 0.1018
Epoch: 006, Loss: 0.0951
Epoch: 007, Loss: 0.0907
Epoch: 008, Loss: 0.0870
Epoch: 009, Loss: 0.0832
Epoch: 010, Loss: 0.0809
Epoch: 011, Loss: 0.0801
Epoch: 012, Loss: 0.0772
Epoch: 013, Loss: 0.0758
Epoch: 014, Loss: 0.0738
Epoch: 015, Loss: 0.0736
Epoch: 016, Loss: 0.0719
Epoch: 017, Loss: 0.0718
Epoch: 018, Loss: 0.0704
Epoch: 019, Loss: 0.0698
Epoch: 020, Loss: 0.0690
Epoch: 021, Loss: 0.0670
Epoch: 022, Loss: 0.0673
Epoch: 023, Loss: 0.0674
Epoch: 024, Loss: 0.0688
Epoch: 025, Loss: 0.0679
Epoch: 026, Loss: 0.0670
Epoch: 027, Loss: 0.0659
Epoch: 028, Loss: 0.0662
Epoch: 029, Loss: 0.0657
Epoch: 030, Loss: 0.0656
Epoch: 031, Loss: 0.0630
Epoch: 032, Loss: 0.0643
Epoch: 033, Loss: 0.0627
Epoch: 034, Loss: 0.0615
Epoch: 035, Loss: 0.0630
Epoch: 036, Loss: 0.0618
Epoch: 037, Loss: 0.0626
Epoch: 038, Loss: 0.0622
Epoch: 039, Loss: 0.0620


**ComplEx Model Results**

In [ ]:
hits1, hits3, hits10, mr, mrr = evaluate(model2,loader2)

In [ ]:
print(f'Mean Rank: {mr:.2f}, Mean Reciprocal Rank: {mrr:.4f}, '
      f'Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}')

Mean Rank: 485.72, Mean Reciprocal Rank: 0.0123, Hits@1: 0.0037, Hits@3: 0.0073, Hits@10: 0.0183


## **Conclusion**

The **ComplEx** model has slightly better MR and MRR scores which means it outperforms it on average (*The Mean Rank is the average rank of the first correct answer - so **lower** values are better, whereas Mean Reciprocial Rank is the average of the reciprocal ranks of the first correct answer for each query which means **higher** numbers are better*).


---


The **ComplEx** model also has higher Hits@3 and Hits@10 than the **ComplEx** model, indicating that it is better at recommending relevant items within the top 3 and top 10 predictions.

# **LAB EXERCISE 4.2**

In [ ]:
!pip install git+https://github.com/pykeen/pykeen.git

  Cloning https://github.com/pykeen/pykeen.git to /tmp/pip-req-build-jb6mednd
  Running command git clone --filter=blob:none --quiet https://github.com/pykeen/pykeen.git /tmp/pip-req-build-jb6mednd
  Resolved https://github.com/pykeen/pykeen.git to commit ac85bef34de3070090012035a2170deddcfb307e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-ma

In [2]:
import matplotlib.pyplot as plt
from pykeen.models import TransE
from pykeen.pipeline import pipeline
from pykeen.datasets import FB15k237, Nations
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator
from pykeen.sampling import BasicNegativeSampler
from torch.optim import Adam

INFO:pykeen.utils:Using opt_einsum


## **EXERCISE 3**

In [2]:
dataset = FB15k237()

In [3]:
model = TransE(triples_factory=dataset.training)
optimizer = Adam(params=model.get_grad_params())
trainer = SLCWATrainingLoop(model=model,
                            triples_factory=dataset.training,
                            negative_sampler=BasicNegativeSampler,
                            optimizer=optimizer)

In [4]:
trainer.train(triples_factory=dataset.training,
                  num_epochs=1,
                  batch_size=64)

Training epochs on cpu:   0%|          | 0/1 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

[0.7436371853580367]

In [5]:
import gc
del trainer
gc.collect()

72

In [6]:
evaluator = RankBasedEvaluator()

In [ ]:
res = evaluator.evaluate(model=model,
                         mapped_triples=dataset.testing.mapped_triples,
                         batch_size=16,
                         additional_filter_triples=[dataset.training.mapped_triples,
                                                    dataset.validation.mapped_triples])

Evaluating on cpu:   0%|          | 0.00/20.4k [00:00<?, ?triple/s]

In [ ]:
res.metric_results.to_df()

## **EXERCISE 4**

In [3]:
dataset = Nations()

In [4]:
dataset.training.entity_id_to_label

{0: 'brazil',
 1: 'burma',
 2: 'china',
 3: 'cuba',
 4: 'egypt',
 5: 'india',
 6: 'indonesia',
 7: 'israel',
 8: 'jordan',
 9: 'netherlands',
 10: 'poland',
 11: 'uk',
 12: 'usa',
 13: 'ussr'}

In [5]:
dataset.training.relation_id_to_label

{0: 'accusation',
 1: 'aidenemy',
 2: 'attackembassy',
 3: 'blockpositionindex',
 4: 'booktranslations',
 5: 'boycottembargo',
 6: 'commonbloc0',
 7: 'commonbloc1',
 8: 'commonbloc2',
 9: 'conferences',
 10: 'dependent',
 11: 'duration',
 12: 'economicaid',
 13: 'eemigrants',
 14: 'embassy',
 15: 'emigrants3',
 16: 'expeldiplomats',
 17: 'exportbooks',
 18: 'exports3',
 19: 'independence',
 20: 'intergovorgs',
 21: 'intergovorgs3',
 22: 'lostterritory',
 23: 'militaryactions',
 24: 'militaryalliance',
 25: 'negativebehavior',
 26: 'negativecomm',
 27: 'ngo',
 28: 'ngoorgs3',
 29: 'nonviolentbehavior',
 30: 'officialvisits',
 31: 'pprotests',
 32: 'relbooktranslations',
 33: 'reldiplomacy',
 34: 'releconomicaid',
 35: 'relemigrants',
 36: 'relexportbooks',
 37: 'relexports',
 38: 'relintergovorgs',
 39: 'relngo',
 40: 'relstudents',
 41: 'reltourism',
 42: 'reltreaties',
 43: 'severdiplomatic',
 44: 'students',
 45: 'timesinceally',
 46: 'timesincewar',
 47: 'tourism',
 48: 'tourism3',


In [6]:
model = TransE(triples_factory=dataset.training)
optimizer = Adam(params=model.get_grad_params())
trainer = SLCWATrainingLoop(model=model,
                            triples_factory=dataset.training,
                            optimizer=optimizer)

In [7]:
trainer.train(triples_factory=dataset.training,
                  num_epochs=2,
                  batch_size=64)

Training epochs on cpu:   0%|          | 0/2 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

[1.413187074661255, 1.3555426836013793]

In [8]:
evaluator = RankBasedEvaluator()
res = evaluator.evaluate(model=model,
                          mapped_triples=dataset.testing.mapped_triples,
                          batch_size=128,
                          additional_filter_triples=[dataset.training.mapped_triples,
                                                    dataset.validation.mapped_triples])

Evaluating on cpu:   0%|          | 0.00/201 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.12s seconds


In [ ]:
from pykeen import predict
preds = predict.predict_target(model=model,
                              head="uk",
                              relation="intergovorgs",
                              triples_factory=dataset.training).filter_triples(dataset.testing).df

In [9]:
from pykeen.predict import predict_all
predict_all(model=model)

scoring:   0%|          | 0.00/770 [00:00<?, ?batch/s]

ScorePack(result=tensor([[12,  3, 12],
        [ 9,  3,  9],
        [ 8,  3,  8],
        ...,
        [ 8, 38, 12],
        [ 6, 54,  9],
        [12, 37,  1]]), scores=tensor([ -5.8881,  -5.8881,  -5.8881,  ..., -12.9004, -13.1130, -13.4276]))

In [10]:
results = predict_all(model=model).result.detach().numpy()

scoring:   0%|          | 0.00/770 [00:00<?, ?batch/s]

In [ ]:
results

array([[11, 41, 11],
       [ 3, 41,  3],
       [ 4, 41,  4],
       ...,
       [ 7, 24, 13],
       [ 9, 37,  2],
       [ 9, 44,  1]])

In [16]:
countries = dataset.testing.entity_to_id
relations = dataset.testing.relation_to_id

## **Get actual relations in the test set for given country**

In [17]:
def get_relations_for_country(country):
    country_id = countries[country]
    country_relations = []
    for h, r, t in dataset.testing.mapped_triples:
        if h == country_id or t == country_id:
            head = list(countries.keys())[list(countries.values()).index(h)]
            tail = list(countries.keys())[list(countries.values()).index(t)]
            relation = list(relations.keys())[list(relations.values()).index(r)]
            country_relations.append((head, relation, tail))
    return country_relations

### **actual UK relations**

In [25]:
uk_actual = get_relations_for_country('uk')
uk_actual

[('brazil', 'embassy', 'uk'),
 ('burma', 'dependent', 'uk'),
 ('china', 'accusation', 'uk'),
 ('cuba', 'intergovorgs3', 'uk'),
 ('cuba', 'negativebehavior', 'uk'),
 ('cuba', 'reltreaties', 'uk'),
 ('egypt', 'commonbloc1', 'uk'),
 ('egypt', 'embassy', 'uk'),
 ('india', 'relexports', 'uk'),
 ('indonesia', 'accusation', 'uk'),
 ('indonesia', 'militaryactions', 'uk'),
 ('indonesia', 'negativecomm', 'uk'),
 ('indonesia', 'reldiplomacy', 'uk'),
 ('indonesia', 'violentactions', 'uk'),
 ('jordan', 'weightedunvote', 'uk'),
 ('netherlands', 'commonbloc2', 'uk'),
 ('netherlands', 'relexportbooks', 'uk'),
 ('netherlands', 'relintergovorgs', 'uk'),
 ('netherlands', 'tourism3', 'uk'),
 ('poland', 'independence', 'uk'),
 ('poland', 'weightedunvote', 'uk'),
 ('uk', 'conferences', 'indonesia'),
 ('uk', 'duration', 'indonesia'),
 ('uk', 'embassy', 'netherlands'),
 ('uk', 'intergovorgs3', 'brazil'),
 ('uk', 'intergovorgs3', 'netherlands'),
 ('uk', 'ngo', 'poland'),
 ('uk', 'pprotests', 'ussr'),
 ('uk', '

### **actual USA relations**

In [26]:
us_actual = get_relations_for_country('usa')
us_actual

[('brazil', 'exports3', 'usa'),
 ('brazil', 'intergovorgs3', 'usa'),
 ('burma', 'blockpositionindex', 'usa'),
 ('burma', 'relintergovorgs', 'usa'),
 ('china', 'intergovorgs3', 'usa'),
 ('cuba', 'independence', 'usa'),
 ('cuba', 'timesincewar', 'usa'),
 ('egypt', 'independence', 'usa'),
 ('egypt', 'ngoorgs3', 'usa'),
 ('egypt', 'reldiplomacy', 'usa'),
 ('india', 'relexports', 'usa'),
 ('india', 'reltreaties', 'usa'),
 ('india', 'students', 'usa'),
 ('indonesia', 'militaryalliance', 'usa'),
 ('indonesia', 'relngo', 'usa'),
 ('indonesia', 'students', 'usa'),
 ('israel', 'intergovorgs3', 'usa'),
 ('israel', 'relngo', 'usa'),
 ('jordan', 'ngoorgs3', 'usa'),
 ('jordan', 'reldiplomacy', 'usa'),
 ('jordan', 'relngo', 'usa'),
 ('netherlands', 'treaties', 'usa'),
 ('poland', 'reldiplomacy', 'usa'),
 ('poland', 'relexports', 'usa'),
 ('poland', 'relintergovorgs', 'usa'),
 ('uk', 'relngo', 'usa'),
 ('uk', 'timesinceally', 'usa'),
 ('usa', 'accusation', 'cuba'),
 ('usa', 'blockpositionindex', 'ussr

## **Get predicted relations for given country**

In [41]:
from pykeen.predict import predict_triples
def predict_relations(country):
    country_id = dataset.training.entity_to_id[country]
    results = predict_all(model=model).result.detach().numpy()

    predicted_relations = set()

    for h, r, t in results:
        if h == country_id or t == country_id:
            head = dataset.training.entity_id_to_label[h]
            tail = dataset.training.entity_id_to_label[t]
            relation = dataset.training.relation_id_to_label[r]
            predicted_relations.add((head, relation, tail))

    return predicted_relations

### **predicted UK relations**

In [42]:
uk_predicted = predict_relations('uk')
uk_predicted

scoring:   0%|          | 0.00/770 [00:00<?, ?batch/s]

{('israel', 'aidenemy', 'uk'),
 ('uk', 'expeldiplomats', 'poland'),
 ('uk', 'boycottembargo', 'netherlands'),
 ('uk', 'timesincewar', 'uk'),
 ('uk', 'militaryalliance', 'ussr'),
 ('egypt', 'attackembassy', 'uk'),
 ('uk', 'commonbloc2', 'poland'),
 ('india', 'relexports', 'uk'),
 ('india', 'severdiplomatic', 'uk'),
 ('uk', 'negativebehavior', 'egypt'),
 ('uk', 'emigrants3', 'india'),
 ('uk', 'treaties', 'egypt'),
 ('indonesia', 'intergovorgs', 'uk'),
 ('netherlands', 'ngo', 'uk'),
 ('uk', 'accusation', 'brazil'),
 ('uk', 'treaties', 'cuba'),
 ('poland', 'eemigrants', 'uk'),
 ('uk', 'treaties', 'indonesia'),
 ('cuba', 'relstudents', 'uk'),
 ('uk', 'boycottembargo', 'brazil'),
 ('china', 'intergovorgs3', 'uk'),
 ('uk', 'reldiplomacy', 'china'),
 ('uk', 'militaryalliance', 'usa'),
 ('uk', 'reltourism', 'poland'),
 ('uk', 'releconomicaid', 'israel'),
 ('uk', 'negativebehavior', 'india'),
 ('uk', 'treaties', 'india'),
 ('brazil', 'timesincewar', 'uk'),
 ('india', 'negativecomm', 'uk'),
 ('ne

### **predicted US relations**

In [43]:
us_predicted = predict_relations('usa')
us_predicted

scoring:   0%|          | 0.00/770 [00:00<?, ?batch/s]

{('brazil', 'lostterritory', 'usa'),
 ('usa', 'negativebehavior', 'egypt'),
 ('usa', 'emigrants3', 'india'),
 ('usa', 'treaties', 'egypt'),
 ('egypt', 'dependent', 'usa'),
 ('netherlands', 'expeldiplomats', 'usa'),
 ('poland', 'embassy', 'usa'),
 ('netherlands', 'commonbloc0', 'usa'),
 ('jordan', 'ngoorgs3', 'usa'),
 ('usa', 'accusation', 'brazil'),
 ('usa', 'relexports', 'ussr'),
 ('netherlands', 'commonbloc2', 'usa'),
 ('usa', 'treaties', 'indonesia'),
 ('indonesia', 'independence', 'usa'),
 ('jordan', 'eemigrants', 'usa'),
 ('usa', 'reldiplomacy', 'china'),
 ('usa', 'boycottembargo', 'brazil'),
 ('netherlands', 'timesincewar', 'usa'),
 ('poland', 'lostterritory', 'usa'),
 ('usa', 'reltourism', 'poland'),
 ('israel', 'nonviolentbehavior', 'usa'),
 ('poland', 'exportbooks', 'usa'),
 ('usa', 'relngo', 'ussr'),
 ('israel', 'expeldiplomats', 'usa'),
 ('uk', 'militaryalliance', 'usa'),
 ('cuba', 'timesincewar', 'usa'),
 ('usa', 'releconomicaid', 'israel'),
 ('usa', 'negativebehavior', 'in

# **CONCLUSION**

**Correctly predicted relations for UK**

In [49]:
print(set(uk_actual) & uk_predicted)

{('indonesia', 'reldiplomacy', 'uk'), ('usa', 'ngoorgs3', 'uk'), ('netherlands', 'tourism3', 'uk'), ('ussr', 'blockpositionindex', 'uk'), ('burma', 'dependent', 'uk'), ('jordan', 'weightedunvote', 'uk'), ('indonesia', 'militaryactions', 'uk'), ('uk', 'timesinceally', 'ussr'), ('uk', 'duration', 'indonesia'), ('indonesia', 'negativecomm', 'uk'), ('uk', 'timesinceally', 'cuba'), ('uk', 'embassy', 'netherlands'), ('uk', 'intergovorgs3', 'netherlands'), ('india', 'relexports', 'uk'), ('indonesia', 'violentactions', 'uk'), ('china', 'accusation', 'uk'), ('netherlands', 'relexportbooks', 'uk'), ('uk', 'conferences', 'indonesia'), ('uk', 'unweightedunvote', 'jordan'), ('netherlands', 'commonbloc2', 'uk'), ('indonesia', 'accusation', 'uk'), ('poland', 'independence', 'uk'), ('poland', 'weightedunvote', 'uk'), ('uk', 'ngo', 'poland'), ('egypt', 'commonbloc1', 'uk'), ('cuba', 'reltreaties', 'uk'), ('uk', 'pprotests', 'ussr'), ('uk', 'relngo', 'usa'), ('uk', 'weightedunvote', 'cuba'), ('cuba', 'i

In [51]:
print(len(set(uk_actual) & uk_predicted))

40


**Correctly predicted relations for USA**

In [45]:
print(set(us_actual) & us_predicted)

{('usa', 'militaryalliance', 'netherlands'), ('usa', 'ngoorgs3', 'uk'), ('usa', 'negativebehavior', 'egypt'), ('ussr', 'booktranslations', 'usa'), ('usa', 'conferences', 'indonesia'), ('cuba', 'independence', 'usa'), ('brazil', 'exports3', 'usa'), ('egypt', 'independence', 'usa'), ('usa', 'reltourism', 'india'), ('poland', 'reldiplomacy', 'usa'), ('jordan', 'ngoorgs3', 'usa'), ('burma', 'blockpositionindex', 'usa'), ('indonesia', 'students', 'usa'), ('india', 'relexports', 'usa'), ('usa', 'weightedunvote', 'ussr'), ('usa', 'weightedunvote', 'cuba'), ('usa', 'unweightedunvote', 'israel'), ('poland', 'relexports', 'usa'), ('india', 'students', 'usa'), ('usa', 'reltourism', 'egypt'), ('burma', 'relintergovorgs', 'usa'), ('usa', 'ngoorgs3', 'netherlands'), ('usa', 'weightedunvote', 'israel'), ('netherlands', 'treaties', 'usa'), ('israel', 'relngo', 'usa'), ('usa', 'blockpositionindex', 'ussr'), ('usa', 'negativecomm', 'brazil'), ('jordan', 'relngo', 'usa'), ('indonesia', 'relngo', 'usa'), 

In [46]:
print(len(set(us_actual) & us_predicted))

51
